In [1]:
try:
    import openmdao.api as om
except ImportError:
    !python -m pip install openmdao[notebooks]
    import openmdao.api as om
    
from openmdao.utils.assert_utils import assert_near_equal

# Case Reader

The `CaseReader` object is provided to read case recordings, regardless of which case recorder was used. 

Currently, OpenMDAO only implements `SqliteCaseRecorder`, therefore all the examples will make use of this recorder. Other types of case recorders are expected to be supported in the future.

## CaseReader Constructor

The call signature for the `CaseReader` constructor is:

```
    .. automethod:: openmdao.recorders.sqlite_reader.SqliteCaseReader.__init__
        :noindex:
```

## Determining What Sources and Variables Were Recorded

The `CaseReader` object provides methods to determine which objects in the original problem were sources
for for the recorded cases and what variables they recorded. Sources can include the problem, driver, components and solvers.

The `list_sources` method provides a list of the names of objects that are the sources of recorded data
in the file.
```
.. automethod:: openmdao.recorders.base_case_reader.BaseCaseReader.list_sources
    :noindex:
```

The complementary `list_source_vars` method will provide a list of the input and output variables recorded
for a given source.

```
.. automethod:: openmdao.recorders.base_case_reader.BaseCaseReader.list_source_vars
    :noindex:
```

Here is an example of their usage:

In [2]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarMDA

import numpy as np

# define Sellar MDA problem
prob = om.Problem(model=SellarMDA())

model = prob.model
model.add_design_var('z', lower=np.array([-10.0, 0.0]),
                          upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9, disp=False)

# add recorder to the driver, model and solver
recorder = om.SqliteRecorder('cases.sql')

prob.driver.add_recorder(recorder)
model.add_recorder(recorder)
model.nonlinear_solver.add_recorder(recorder)

# run the problem
prob.setup()
prob.set_solver_print(0)
prob.run_driver()
prob.cleanup()

cr = om.CaseReader('cases.sql')

In [3]:
sources = cr.list_sources()

Sources
driver
root.nonlinear_solver
root


In [4]:
assert sorted(sources) == ['driver', 'root', 'root.nonlinear_solver']

In [5]:
driver_vars = cr.list_source_vars('driver')

inputs,outputs,residuals
,z,
,con1,
,x,
,con2,
,obj,


In [6]:
assert driver_vars['inputs'] == []
assert set(driver_vars['outputs']) == {'con1', 'con2', 'obj', 'x', 'z'}
assert driver_vars['residuals'] == []

In [7]:
model_vars = cr.list_source_vars('root')

inputs,outputs,residuals
y1,z,z
y2,x,x
x,con1,con1
z,con2,con2
,y1,y1
,y2,y2
,obj,obj


In [8]:
assert set(model_vars['inputs']) == {'x', 'y1', 'y2', 'z'}
assert set(model_vars['outputs']) == {'con1', 'con2', 'obj', 'x', 'y1', 'y2', 'z'}
assert set(model_vars['residuals']) == {'con1', 'con2', 'obj', 'x', 'y1', 'y2', 'z'}

In [9]:
solver_vars = cr.list_source_vars('root.nonlinear_solver')

inputs,outputs,residuals
z,z,
x,x,
y2,y1,
y1,y2,
,obj,
,con1,
,con2,


In [10]:
assert set(solver_vars['inputs']) == {'x', 'y1', 'y2', 'z'}
assert set(solver_vars['outputs']) == {'con1', 'con2', 'obj', 'x', 'y1', 'y2', 'z'}
assert solver_vars['residuals'] == []

## Case Names

The `CaseReader` provides access to `Case` objects, each of which encapsulates a data point recorded by
one of the sources.

`Case` objects are uniquely identified in a case recorder file by their case names. A case name is a string.
As an example, here is a case name:

    'rank0:ScipyOptimize_SLSQP|1|root._solve_nonlinear|1'

The first part of the case name indicates which rank or process that the case was recorded from. 
The remainder of the case name shows the hierarchical path to the object that was recorded along 
with the iteration counts for each object along the path. It follows a pattern of repeated pairs of

    - object name ( problem, driver, system, or solver )
    - iteration count

These are separated by the `|` character.

So in the given example, the case is:

    - from rank 0
    - the first iteration of the driver, `ScipyOptimize_SLSQP`
    - the first execution of the `root` system which is the top-level model


## Getting Names of the Cases

The `list_cases` method returns the names of the cases in the order in which
the cases were executed. You can optionally request cases only from a specific `source`.

```
.. automethod:: openmdao.recorders.base_case_reader.BaseCaseReader.list_cases
    :noindex:
```

There are two optional arguments to the `list_cases` method that affect what is returned.

    - recurse: causes the returned value to include child cases.

    - flat: works in conjunction with the `recurse` argument to determine if the returned
      results are in the form of a list or nested dict. If recurse=True, flat=False, and there
      are child cases, then the returned value is a nested ordered dict. Otherwise, it is a list.



## Accessing Cases

Getting information from the cases is a two-step process. First, you need to get access to the Case object and then you can call a variety of methods on the Case object to get values from it. The second step is described on the :ref:`Accessing Recorded Data<reading_case_data>` page.

There are two methods used to get a specific `Case`:

    - get_cases
    - get_case



### Accessing Cases Using get_cases Method

The `get_cases` method provides a quick and easy way to iterate over all the cases.
```
.. automethod:: openmdao.recorders.base_case_reader.BaseCaseReader.get_cases
    :noindex:
```
This method is similar to the `list_cases` method in that it has the two optional arguments
`recurse` and `flat` to control what is returned as described above.

Here is an example of its usage:

In [11]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarMDA

import numpy as np

prob = om.Problem(model=SellarMDA())

model = prob.model
model.add_design_var('z', lower=np.array([-10.0, 0.0]),
                          upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

driver = prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9)
driver.add_recorder(om.SqliteRecorder('cases.sql'))

prob.setup()
prob.set_solver_print(0)
prob.run_driver()
prob.cleanup()

Optimization terminated successfully    (Exit mode 0)
            Current function value: [3.18339395]
            Iterations: 12
            Function evaluations: 13
            Gradient evaluations: 12
Optimization Complete
-----------------------------------


In [12]:
cr = om.CaseReader('cases.sql')
cases = cr.get_cases()
for case in cases:
    print(case.name, sorted(case.outputs))

rank0:ScipyOptimize_SLSQP|0 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|1 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|2 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|3 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|4 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|5 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|6 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|7 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|8 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|9 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|10 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|11 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|12 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|13 ['con1', 'con2', 'obj', 'x', 'z']


In [13]:
assert len(cases) == driver.iter_count
for i, case in enumerate(cases):
    assert case.name == f"rank0:ScipyOptimize_SLSQP|{i}"
    assert sorted(case.outputs) == ['con1', 'con2', 'obj', 'x', 'z']

In [14]:
assert len(cases) == driver.iter_count
for i, case in enumerate(cases):
    assert case.name == f"rank0:ScipyOptimize_SLSQP|{i}"
    assert sorted(case.outputs) == ['con1', 'con2', 'obj', 'x', 'z']


### Accessing Cases Using get_case Method

The `get_case` method returns a `Case` object given a case name.
```
.. automethod:: openmdao.recorders.base_case_reader.BaseCaseReader.get_case
    :noindex:
```
You can use the `get_case` method to get a specific case from the list of case names
returned by `list_cases` as shown here:

In [15]:
    case_names = cr.list_cases()

driver
rank0:ScipyOptimize_SLSQP|0
rank0:ScipyOptimize_SLSQP|1
rank0:ScipyOptimize_SLSQP|2
rank0:ScipyOptimize_SLSQP|3
rank0:ScipyOptimize_SLSQP|4
rank0:ScipyOptimize_SLSQP|5
rank0:ScipyOptimize_SLSQP|6
rank0:ScipyOptimize_SLSQP|7
rank0:ScipyOptimize_SLSQP|8
rank0:ScipyOptimize_SLSQP|9


In [16]:
assert len(case_names) == driver.iter_count
for i, case_name in enumerate(case_names):
    assert case_name == f"rank0:ScipyOptimize_SLSQP|{i}"

In [17]:
# access a Case by name (e.g. first case)
case = cr.get_case("rank0:ScipyOptimize_SLSQP|0")
print(case.name, sorted(case.outputs))

rank0:ScipyOptimize_SLSQP|0 ['con1', 'con2', 'obj', 'x', 'z']


In [18]:
assert case.name == "rank0:ScipyOptimize_SLSQP|0"

In [19]:
# access a Case by index (e.g. first case)
case = cr.get_case(0)
print(case.name, sorted(case.outputs))

rank0:ScipyOptimize_SLSQP|0 ['con1', 'con2', 'obj', 'x', 'z']


In [20]:
assert case.name == "rank0:ScipyOptimize_SLSQP|0"

In [21]:
# access a Case by index (e.g. last case)
case = cr.get_case(-1)
print(case.name, sorted(case.outputs))

rank0:ScipyOptimize_SLSQP|13 ['con1', 'con2', 'obj', 'x', 'z']


In [22]:
assert case.name == "rank0:ScipyOptimize_SLSQP|13"

In [25]:
# get each case by looping over case names
for name in case_names:
    case = cr.get_case(name)
    print(case.name, sorted(case.outputs))

rank0:ScipyOptimize_SLSQP|0 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|1 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|2 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|3 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|4 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|5 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|6 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|7 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|8 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|9 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|10 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|11 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|12 ['con1', 'con2', 'obj', 'x', 'z']
rank0:ScipyOptimize_SLSQP|13 ['con1', 'con2', 'obj', 'x', 'z']


### Processing a Nested Dictionary of Its Child Cases

The following example demonstrates selecting a case from a case list and processing a nested
dictionary of its child cases.

In [27]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarMDA

import numpy as np

# define Sellar MDA problem
prob = om.Problem(model=SellarMDA())

model = prob.model
model.add_design_var('z', lower=np.array([-10.0, 0.0]),
                          upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9, disp=False)

# add recorder to the driver, model and solver
recorder = om.SqliteRecorder('cases.sql')

prob.driver.add_recorder(recorder)
model.add_recorder(recorder)
model.nonlinear_solver.add_recorder(recorder)

# run the problem
prob.setup()
prob.set_solver_print(0)
prob.run_driver()
prob.cleanup()

In [56]:
cr = om.CaseReader('cases.sql')

# get the last driver case
driver_cases = cr.list_cases('driver')

solver
rank0:ScipyOptimize_SLSQP|0|root._solve_nonlinear|0|NLRunOnce|0


system
rank0:ScipyOptimize_SLSQP|0|root._solve_nonlinear|0


driver
rank0:ScipyOptimize_SLSQP|0


solver
rank0:ScipyOptimize_SLSQP|1|root._solve_nonlinear|1|NLRunOnce|0


system
rank0:ScipyOptimize_SLSQP|1|root._solve_nonlinear|1


...

system
rank0:ScipyOptimize_SLSQP|12|root._solve_nonlinear|12


driver
rank0:ScipyOptimize_SLSQP|12


solver
rank0:ScipyOptimize_SLSQP|13|root._solve_nonlinear|13|NLRunOnce|0


system
rank0:ScipyOptimize_SLSQP|13|root._solve_nonlinear|13


driver
rank0:ScipyOptimize_SLSQP|13


In [55]:
# get a recursive dict of all child cases of the last driver case
last_driver_case = driver_cases[-1]
cases = cr.get_cases(last_driver_case, recurse=True, flat=False)

for case in cases:
    print(case.name, case.source, '\n', sorted(case.outputs))
    for child_case in cases[case]:
            print('   ', child_case.name, child_case.source, '\n   ', sorted(child_case.outputs))
            for grandchild in cases[case][child_case]:
                print('      ', grandchild.name, grandchild.source, '\n      ', sorted(grandchild.outputs))

rank0:ScipyOptimize_SLSQP|13 driver 
 ['con1', 'con2', 'obj', 'x', 'z']
    rank0:ScipyOptimize_SLSQP|13|root._solve_nonlinear|13 root 
    ['con1', 'con2', 'obj', 'x', 'y1', 'y2', 'z']
       rank0:ScipyOptimize_SLSQP|13|root._solve_nonlinear|13|NLRunOnce|0 root.nonlinear_solver 
       ['con1', 'con2', 'obj', 'x', 'y1', 'y2', 'z']
